# 커뮤니티 파이프라인

> **커뮤니티 파이프라인에 대한 자세한 내용은 [이 이슈](https://github.com/huggingface/diffusers/issues/841)를 참조하십시오.**

**커뮤니티** 예제는 커뮤니티에서 추가한 추론 및 학습 예제로 구성됩니다.
모든 커뮤니티 예제에 대한 개요는 다음 표를 참조하십시오. **코드 예제**를 클릭하면 복사하여 붙여넣을 수 있는 코드 예제를 얻을 수 있습니다.
커뮤니티가 예상대로 작동하지 않으면 이슈를 열고 작성자에게 핑을 보내십시오.

| 예제                                | 설명                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | 코드 예제                                                      | Colab                                                                                                                                                                                                              |                                                     작성자 |
|:---------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:------------------------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|-----------------------------------------------------------:|
| CLIP Guided Stable Diffusion           | Stable Diffusion을 사용한 텍스트-이미지 생성을 위한 CLIP 안내 수행                                                                                                                                                                                                                                                                                                                                                                                                                                   | [CLIP Guided Stable Diffusion](#clip-guided-stable-diffusion)     | [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/CLIP_Guided_Stable_diffusion_with_diffusers.ipynb) |             [Suraj Patil](https://github.com/patil-suraj/) |
| One Step U-Net (Dummy)                 | 커뮤니티 파이프라인 사용 방법을 보여주는 예제 (https://github.com/huggingface/diffusers/issues/841 참조)                                                                                                                                                                                                                                                                                                                                                                                           | [One Step U-Net](#one-step-unet)                                  | -                                                                                                                                                                                                                  | [Patrick von Platen](https://github.com/patrickvonplaten/) |
| Stable Diffusion Interpolation         | 다른 프롬프트/시드 간에 Stable Diffusion의 잠재 공간 보간                                                                                                                                                                                                                                                                                                                                                                                                                         | [Stable Diffusion Interpolation](#stable-diffusion-interpolation) | -                                                                                                                                                                                                                  |                    [Nate Raw](https://github.com/nateraw/) |
| Stable Diffusion Mega                  | [Text2Image](https://github.com/huggingface/diffusers/blob/main/src/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion.py), [Image2Image](https://github.com/huggingface/diffusers/blob/main/src/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion_img2img.py) 및 [Inpainting](https://github.com/huggingface/diffusers/blob/main/src/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion_inpaint.py)의 모든 기능을 갖춘 **하나의** Stable Diffusion 파이프라인 | [Stable Diffusion Mega](#stable-diffusion-mega)                   | -                                                                                                                                                                                                                  | [Patrick von Platen](https://github.com/patrickvonplaten/) |
| Long Prompt Weighting Stable Diffusion | 토큰 길이 제한이 없고 프롬프트에서 가중치 구문 분석을 지원하는 **하나의** Stable Diffusion 파이프라인.                                                                                                                                                                                                                                                                                                                                                                                                  | [Long Prompt Weighting Stable Diffusion](#long-prompt-weighting-stable-diffusion)                                                                 | -                                                                                                                                                                                                                  |                        [SkyTNT](https://github.com/SkyTNT) |
| Speech to Image                        | 자동 음성 인식을 사용하여 텍스트를 기록하고 Stable Diffusion을 사용하여 이미지 생성                                                                                                                                                                                                                                                                                                                                                                                                            | [Speech to Image](#speech-to-image)                               | -                                                                                                                                                                                                                  | [Mikail Duzenli](https://github.com/MikailINTech)

사용자 지정 파이프라인을 로드하려면 `diffusers/examples/community`의 파일 중 하나로 `custom_pipeline` 인수를 `DiffusionPipeline`에 전달하기만 하면 됩니다. 자체 파이프라인으로 PR을 자유롭게 보내주십시오. 신속하게 병합해 드리겠습니다.

In [ ]:
pipe = DiffusionPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4", custom_pipeline="filename_in_the_community_folder"
)

## 사용 예시

### CLIP 가이드 스테이블 디퓨전

CLIP 가이드 스테이블 디퓨전은 추가 CLIP 모델을 사용하여 모든 노이즈 제거 단계에서 스테이블 디퓨전을 안내하여 보다 사실적인 이미지를 생성하는 데 도움이 될 수 있습니다.

다음 코드는 약 12GB의 GPU RAM이 필요합니다.

In [ ]:
from diffusers import DiffusionPipeline
from transformers import CLIPImageProcessor, CLIPModel
import torch


feature_extractor = CLIPImageProcessor.from_pretrained("laion/CLIP-ViT-B-32-laion2B-s34B-b79K")
clip_model = CLIPModel.from_pretrained("laion/CLIP-ViT-B-32-laion2B-s34B-b79K", torch_dtype=torch.float16)


guided_pipeline = DiffusionPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    custom_pipeline="clip_guided_stable_diffusion",
    clip_model=clip_model,
    feature_extractor=feature_extractor,
    torch_dtype=torch.float16,
)
guided_pipeline.enable_attention_slicing()
guided_pipeline = guided_pipeline.to("cuda")

prompt = "fantasy book cover, full moon, fantasy forest landscape, golden vector elements, fantasy magic, dark light night, intricate, elegant, sharp focus, illustration, highly detailed, digital painting, concept art, matte, art by WLOP and Artgerm and Albert Bierstadt, masterpiece"

generator = torch.Generator(device="cuda").manual_seed(0)
images = []
for i in range(4):
    image = guided_pipeline(
        prompt,
        num_inference_steps=50,
        guidance_scale=7.5,
        clip_guidance_scale=100,
        num_cutouts=4,
        use_cutouts=False,
        generator=generator,
    ).images[0]
    images.append(image)

# 로컬에 이미지 저장
for i, img in enumerate(images):
    img.save(f"./clip_guided_sd/image_{i}.png")

`images` 목록에는 로컬에 저장하거나 Google Colab에 직접 표시할 수 있는 PIL 이미지 목록이 포함되어 있습니다.
생성된 이미지는 기본적으로 안정적인 확산을 사용하는 것보다 품질이 높은 경향이 있습니다. 예를 들어 위의 스크립트는 다음 이미지를 생성합니다.

![clip_guidance](https://huggingface.co/datasets/patrickvonplaten/images/resolve/main/clip_guidance/merged_clip_guidance.jpg).

### 원스텝 유넷

더미 "one-step-unet"은 다음과 같이 실행할 수 있습니다.

In [ ]:
from diffusers import DiffusionPipeline

pipe = DiffusionPipeline.from_pretrained("google/ddpm-cifar10-32", custom_pipeline="one_step_unet")
pipe()

**참고**: 이 커뮤니티 파이프라인은 기능으로 유용하지 않지만 커뮤니티 파이프라인을 추가하는 방법을 보여주는 예제로만 사용됩니다(https://github.com/huggingface/diffusers/issues/841 참조).

### 스테이블 디퓨전 보간

다음 코드는 최소 8GB VRAM의 GPU에서 실행할 수 있으며 약 5분이 소요됩니다.

In [ ]:
from diffusers import DiffusionPipeline
import torch

pipe = DiffusionPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    torch_dtype=torch.float16,
    safety_checker=None,  # 동영상에는 매우 중요합니다... 보간 중에 많은 오탐이 발생합니다.
    custom_pipeline="interpolate_stable_diffusion",
).to("cuda")
pipe.enable_attention_slicing()

frame_filepaths = pipe.walk(
    prompts=["a dog", "a cat", "a horse"],
    seeds=[42, 1337, 1234],
    num_interpolation_steps=16,
    output_dir="./dreams",
    batch_size=4,
    height=512,
    width=512,
    guidance_scale=8.5,
    num_inference_steps=50,
)

`walk(...)` 함수의 출력은 `output_dir`에 정의된 폴더 아래에 저장된 이미지 목록을 반환합니다. 이러한 이미지를 사용하여 안정적인 확산 동영상을 만들 수 있습니다.

> **안정적인 확산을 사용하여 동영상을 만드는 방법에 대한 자세한 정보와 보다 완전한 기능은 https://github.com/nateraw/stable-diffusion-videos를 참조하십시오.**

### 스테이블 디퓨전 메가

Stable Diffusion Mega Pipeline을 사용하면 단일 클래스에서 안정적인 확산 파이프라인의 주요 사용 사례를 사용할 수 있습니다.

In [ ]:
#!/usr/bin/env python3
from diffusers import DiffusionPipeline
import PIL
import requests
from io import BytesIO
import torch


def download_image(url):
    response = requests.get(url)
    return PIL.Image.open(BytesIO(response.content)).convert("RGB")


pipe = DiffusionPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    custom_pipeline="stable_diffusion_mega",
    torch_dtype=torch.float16,
)
pipe.to("cuda")
pipe.enable_attention_slicing()


### 텍스트-이미지 변환

images = pipe.text2img("An astronaut riding a horse").images

### 이미지-이미지 변환

init_image = download_image(
    "https://raw.githubusercontent.com/CompVis/stable-diffusion/main/assets/stable-samples/img2img/sketch-mountains-input.jpg"
)

prompt = "A fantasy landscape, trending on artstation"

images = pipe.img2img(prompt=prompt, image=init_image, strength=0.75, guidance_scale=7.5).images

### 인페인팅

img_url = "https://raw.githubusercontent.com/CompVis/latent-diffusion/main/data/inpainting_examples/overture-creations-5sI6fQgYIuo.png"
mask_url = "https://raw.githubusercontent.com/CompVis/latent-diffusion/main/data/inpainting_examples/overture-creations-5sI6fQgYIuo_mask.png"
init_image = download_image(img_url).resize((512, 512))
mask_image = download_image(mask_url).resize((512, 512))

prompt = "a cat sitting on a bench"
images = pipe.inpaint(prompt=prompt, image=init_image, mask_image=mask_image, strength=0.75).images

위에서 보듯이 이 파이프라인 하나로 "텍스트-이미지", "이미지-이미지", "인페인팅"을 모두 실행할 수 있습니다.

### 긴 프롬프트 가중치 안정 확산

이 파이프라인은 77 토큰 길이 제한 없이 프롬프트를 입력할 수 있게 해줍니다. 그리고 "()"를 사용하여 단어 가중치를 높이거나 "[]"를 사용하여 단어 가중치를 낮출 수 있습니다.
또한 이 파이프라인은 단일 클래스에서 안정 확산 파이프라인의 주요 사용 사례를 사용할 수 있게 해줍니다.

#### 파이토치

In [ ]:
from diffusers import DiffusionPipeline
import torch

pipe = DiffusionPipeline.from_pretrained(
    "hakurei/waifu-diffusion", custom_pipeline="lpw_stable_diffusion", torch_dtype=torch.float16
)
pipe = pipe.to("cuda")

prompt = "best_quality (1girl:1.3) bow bride brown_hair closed_mouth frilled_bow frilled_hair_tubes frills (full_body:1.3) fox_ear hair_bow hair_tubes happy hood japanese_clothes kimono long_sleeves red_bow smile solo tabi uchikake white_kimono wide_sleeves cherry_blossoms"
neg_prompt = "lowres, bad_anatomy, error_body, error_hair, error_arm, error_hands, bad_hands, error_fingers, bad_fingers, missing_fingers, error_legs, bad_legs, multiple_legs, missing_legs, error_lighting, error_shadow, error_reflection, text, error, extra_digit, fewer_digits, cropped, worst_quality, low_quality, normal_quality, jpeg_artifacts, signature, watermark, username, blurry"

pipe.text2img(prompt, negative_prompt=neg_prompt, width=512, height=512, max_embeddings_multiples=3).images[0]

#### onnxruntime

In [ ]:
from diffusers import DiffusionPipeline
import torch

pipe = DiffusionPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    custom_pipeline="lpw_stable_diffusion_onnx",
    revision="onnx",
    provider="CUDAExecutionProvider",
)

prompt = "a photo of an astronaut riding a horse on mars, best quality"
neg_prompt = "lowres, bad anatomy, error body, error hair, error arm, error hands, bad hands, error fingers, bad fingers, missing fingers, error legs, bad legs, multiple legs, missing legs, error lighting, error shadow, error reflection, text, error, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry"

pipe.text2img(prompt, negative_prompt=neg_prompt, width=512, height=512, max_embeddings_multiples=3).images[0]

`Token indices sequence length is longer than the specified maximum sequence length for this model ( *** > 77 ) . Running this sequence through the model will result in indexing errors`라는 메시지가 표시되어도 걱정하지 마십시오. 정상적인 현상입니다.

### 음성-이미지 변환

다음 코드는 사전 학습된 OpenAI whisper-small 및 Stable Diffusion을 사용하여 오디오 샘플에서 이미지를 생성할 수 있습니다.

```Python
import torch

import matplotlib.pyplot as plt
from datasets import load_dataset
from diffusers import DiffusionPipeline
from transformers import (
    WhisperForConditionalGeneration,
    WhisperProcessor,
)


device = "cuda" if torch.cuda.is_available() else "cpu"

ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")

audio_sample = ds[3]

text = audio_sample["text"].lower()
speech_data = audio_sample["audio"]["array"]

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small").to(device)
processor = WhisperProcessor.from_pretrained("openai/whisper-small")

diffuser_pipeline = DiffusionPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    custom_pipeline="speech_to_image_diffusion",
    speech_model=model,
    speech_processor=processor,
    
    torch_dtype=torch.float16,
)

diffuser_pipeline.enable_attention_slicing()
diffuser_pipeline = diffuser_pipeline.to(device)

output = diffuser_pipeline(speech_data)
plt.imshow(output.images[0])
```
이 예제는 다음 이미지를 생성합니다.

![image](https://user-images.githubusercontent.com/45072645/196901736-77d9c6fc-63ee-4072-90b0-dc8b903d63e3.png)